In [ ]:
import tensorflow as tf
import numpy as np
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
from pydub import AudioSegment
import shutil
from keras.preprocessing.image import ImageDataGenerator
import random
import json
from PIL import Image
from keras.models import Sequential
import tkinter as tk
from PIL import ImageTk

In [2]:
def create_melspectrogram(wav_file):  
  y,sr = librosa.load(wav_file)  
  mels = librosa.feature.melspectrogram(y=y,sr=sr)    
  fig = plt.Figure()  
  canvas = FigureCanvas(fig) 
  p = plt.imshow(librosa.power_to_db(mels,ref=np.max))   
  plt.savefig('melspectrogram.png')

In [3]:
def predict(image_data,model):   
  image = tf.keras.utils.img_to_array(image_data)   
  image = np.reshape(image,(1,288,432,4))   
  prediction =  model.predict(image/255)  
  prediction = prediction.reshape((10,))     
  class_label = np.argmax(prediction)     
  return class_label,prediction
class_labels = ['blues', 'classic ', 'country', 'disco', 'hiphop','jazz', 'metal', 'pop', 'reggae', 'rock'] 

In [4]:
def record_audio():
    import sounddevice as sd
    from scipy.io.wavfile import write
    import wavio as wv
  
    
    # Sampling frequency
    freq = 44100
  
    # Recording duration
    duration = 3
  
    # Start recorder with the given values 
    # of duration and sample frequency
    recording = sd.rec(int(duration * freq), 
                   samplerate=freq, channels=2)
  
    # Record audio for the given number of seconds
    sd.wait()
  
    # This will convert the NumPy array to an audio
    # file with the given sampling frequency
    write("recording0.wav", freq, recording)
  
    # Convert the NumPy array to audio file
    wv.write("recording1.wav", recording, freq, sampwidth=2)
    create_melspectrogram("recording1.wav")
    image_data = tf.keras.utils.load_img('melspectrogram.png',color_mode='rgba',target_size=(288,432))
    class_label,prediction = predict(image_data,model)
    fig = plt.figure()
    ax = fig.add_axes([0.1,0.1,0.89,0.8])
    ax.bar(class_labels,prediction)
    fig.savefig('prediction.png')

In [5]:
def show_image(imagefile):
    image = ImageTk.PhotoImage(file=imagefile)
    imagebox.config(image=image)
    imagebox.image = image

In [6]:
model=load_model('C:\FinalProject\model.h5',compile = False)

In [ ]:
from tkinter import *
window = Tk()
window.title("Music Genre Classification")
window.geometry("500x400")
app =Frame(window)
app.pack()
button1 = Button(app, text = " Record Audio " , width=20, command=record_audio)
button1.pack(side=tk.LEFT)

button2 = Button(app, text = " Show Prediction " , width=20, command=lambda: show_image("prediction.png"))
button2.pack(side=tk.LEFT)


    
imagebox = tk.Label(window)
imagebox.pack()
 
window.mainloop()